In [1]:
import boto3
import pandas as pd

In [2]:
!pip install python-dotenv

In [3]:
from dotenv import dotenv_values
config = dotenv_values(".env")

In [4]:
access_key = config['ACCESS_KEY']
secret_key = config['SECRET_ACCESS_KEY']

s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)

bucket_name = 'mindex-data-analytics-code-challenge'

s3_obj = s3.get_object(Bucket=bucket_name, Key='bengals.csv')
bengals_df = pd.read_csv(s3_obj['Body'])

s3_obj = s3.get_object(Bucket=bucket_name, Key='boyd_receiving.csv')
boyd_df = pd.read_csv(s3_obj['Body'])

s3_obj = s3.get_object(Bucket=bucket_name, Key='chase_receiving.csv')
chase_df = pd.read_csv(s3_obj['Body'])

s3_obj = s3.get_object(Bucket=bucket_name, Key='higgins_receiving.csv')
higgins_df = pd.read_csv(s3_obj['Body'])

print("DataFrame for bengals.csv:")
print(bengals_df.shape)
print(bengals_df)

print("\nDataFrame for boyd_receiving.csv:")
print(boyd_df.shape)
print(boyd_df)

print("\nDataFrame for chase_receiving.csv:")
print(chase_df.shape)
print(chase_df)

print("\nDataFrame for higgins_receiving.csv:")
print(higgins_df.shape)
print(higgins_df)


DataFrame for bengals.csv:
(25, 4)
     Week Opponent Location  Result
0    PRE1       TB     Away     1.0
1    PRE2      WSH     Away     0.0
2    PRE3      MIA     Home     0.0
3    REG1      MIN     Home     1.0
4    REG2      CHI     Away     0.0
5    REG3      PIT     Away     1.0
6    REG4      JAX     Home     1.0
7    REG5       GB     Home     0.0
8    REG6      DET     Away     1.0
9    REG7      BAL     Away     1.0
10   REG8      NYJ     Away     0.0
11   REG9      CLE     Home     0.0
12  REG10      NaN      NaN     NaN
13  REG11       LV     Away     1.0
14  REG12      PIT     Home     1.0
15  REG13      LAC     Home     0.0
16  REG14       SF     Home     0.0
17  REG15      DEN     Away     1.0
18  REG16      BAL     Home     1.0
19  REG17       KC     Home     1.0
20  REG18      CLE     Away     0.0
21  POST1       LV     Home     1.0
22  POST2      TEN     Away     1.0
23  POST3       KC     Away     1.0
24  POST4      LAR  Neutral     0.0

DataFrame for boyd_receiving

In [5]:
# Merge each data frame with bengals_df and rename new columns accordingly
merged_df = pd.merge(bengals_df, boyd_df, on='Week', how='left')
merged_df = merged_df.rename(columns={'Yards': 'Boyd_Yards', 'TD': 'Boyd_TD'})
merged_df = pd.merge(merged_df,chase_df , on='Week', how='left')
merged_df = merged_df.rename(columns={'Yards': 'Chase_Yards', 'TD': 'Chase_TD'})
merged_df = pd.merge(merged_df,higgins_df, on='Week', how='left')
merged_df = merged_df.rename(columns={'Yards': 'Higgins_Yards', 'TD': 'Higgins_TD'})

print(merged_df)

     Week Opponent Location  Result  Boyd_Yards  Boyd_TD  Chase_Yards  \
0    PRE1       TB     Away     1.0         NaN      NaN          NaN   
1    PRE2      WSH     Away     0.0         NaN      NaN          NaN   
2    PRE3      MIA     Home     0.0         NaN      NaN          NaN   
3    REG1      MIN     Home     1.0        32.0      0.0        101.0   
4    REG2      CHI     Away     0.0        73.0      0.0         54.0   
5    REG3      PIT     Away     1.0        36.0      1.0         65.0   
6    REG4      JAX     Home     1.0       118.0      0.0         77.0   
7    REG5       GB     Home     0.0        24.0      0.0        159.0   
8    REG6      DET     Away     1.0         7.0      0.0         97.0   
9    REG7      BAL     Away     1.0        39.0      0.0        201.0   
10   REG8      NYJ     Away     0.0        69.0      1.0         32.0   
11   REG9      CLE     Home     0.0        11.0      0.0         49.0   
12  REG10      NaN      NaN     NaN         NaN    

In [6]:
# Convert the 'Result' column to string
merged_df['Result'] = merged_df['Result'].astype(str)

# Replace '1.0' with 'Win' and '0.0' with 'Loss' in the 'Result' column
merged_df['Result'] = merged_df['Result'].replace({'1.0': 'Win', '0.0': 'Loss'})

print(merged_df)

     Week Opponent Location Result  Boyd_Yards  Boyd_TD  Chase_Yards  \
0    PRE1       TB     Away    Win         NaN      NaN          NaN   
1    PRE2      WSH     Away   Loss         NaN      NaN          NaN   
2    PRE3      MIA     Home   Loss         NaN      NaN          NaN   
3    REG1      MIN     Home    Win        32.0      0.0        101.0   
4    REG2      CHI     Away   Loss        73.0      0.0         54.0   
5    REG3      PIT     Away    Win        36.0      1.0         65.0   
6    REG4      JAX     Home    Win       118.0      0.0         77.0   
7    REG5       GB     Home   Loss        24.0      0.0        159.0   
8    REG6      DET     Away    Win         7.0      0.0         97.0   
9    REG7      BAL     Away    Win        39.0      0.0        201.0   
10   REG8      NYJ     Away   Loss        69.0      1.0         32.0   
11   REG9      CLE     Home   Loss        11.0      0.0         49.0   
12  REG10      NaN      NaN    nan         NaN      NaN         

In [7]:
!pip install psycopg2

In [8]:
import psycopg2
from sqlalchemy import create_engine


host = config['POSTGRES_HOST']
port = '5432'
database = 'postgres'
username = config['POSTGRES_USERNAME']
password = config['POSTGRES_PASSWORD']
table_name = config['TABLE_NAME']

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

merged_df.to_sql(table_name, engine, if_exists='replace', index=False)

engine.dispose()


